In [1]:
%load_ext line_profiler
import numpy as np
from time import time
import matplotlib.pyplot as plt
# plt.rcParams['figure.figsize'] = [10, 10] # 10 x 10 inches plot

In [2]:
# r = np.random.random((12, 16, 16))
# r = np.load('../3d_numpy_array_reduced-58-64-64.npy')#[::4,::4,::4]
k = 4
r = np.load('../cube-full-460-512-512.npy')[::k,::k,::k]
# r = np.load('../3d_numpy_array_reduced-58-64-64.npy')
ro  = 1e-5 + 1.24e-3*r - 2.83e-7*r*r + 2.79e-11*r*r*r
c = (ro + 0.112) * 1.38e-6

# тут не факт что все 1
l = 1 # dt
h = 1 # dx = dy = dz
K = l / h * c
K2 = K**2
K_2_by_3 = K**2 / 3
K2_3_ro = K2 / 3 / ro


# initial conditions
# P_pp = np.zeros_like(ro) # previous previous t - 2
# P_p  = np.zeros_like(ro) # previous          t - 1
# P    = np.zeros_like(ro) # current           t

N = 16
S = 16 # first test simple case when N == S
P_pp = np.random.random(ro.shape) # previous previous t - 2
P_p  = np.random.random(ro.shape) # previous          t - 1
P    = np.zeros(ro.shape, dtype=np.float64) # current           t

In [3]:
def P_step_ravel(P_pp, P_p):
    '''
    mb work with flat and then reshape in return
    norm by now, mb add some more optimisations in future, also cuda
    '''

    S = P_p.shape[0]
    N = P_p.shape[1]
    
    P = np.zeros_like(P_p)
    P[2:-2, 2:-2, 2:-2] = 2 * P_p[2:-2, 2:-2, 2:-2] - P_pp[2:-2, 2:-2, 2:-2]
    Z = np.zeros_like(P_p)
    Z[2:-2, 2:-2, 2:-2] = 22.5 * P_p[2:-2, 2:-2, 2:-2]
    
    cell_indeces_flat = np.arange(S * N * N).reshape(S, N, N)[2:-2, 2:-2, 2:-2].ravel().reshape(-1, 1) # vertical vector

    s1_indexes_flat = cell_indeces_flat + np.array([-1, 1, -N, N, -N**2, N**2])      # i±1 j±1 k±1 
    s2_indexes_flat = cell_indeces_flat + np.array([-1, 1, -N, N, -N**2, N**2]) * 2  # i±2 j±2 k±2 
    s1_values = P_p.ravel()[s1_indexes_flat] # each row contains 6 neighbors of cell 
    s2_values = P_p.ravel()[s2_indexes_flat] # each row contains 6 neighbors of cell 
    s1 = np.sum(s1_values, axis=1) # sum by axis=1 is faster for default order
    s2 = np.sum(s2_values, axis=1)

    Z[2:-2, 2:-2, 2:-2] -=   4 * s1.reshape(S-4, N-4, N-4)
    Z[2:-2, 2:-2, 2:-2] += 1/4 * s2.reshape(S-4, N-4, N-4)

    m1 = np.array([1, -1, -1/8, -1/8])
    m2 = np.array([1, -1])

    s3_V_indexes = cell_indeces_flat + np.array([N**2, -N**2, 2*N**2, -2*N**2])
    s3_V_values = P_p.ravel()[s3_V_indexes] * m1 # po idee mozhno za skobki kak to vinesti m1 i m2
    s3_V_sum = np.sum(s3_V_values, axis=1)
    s3_N_indexes = cell_indeces_flat + np.array([N**2, -N**2])
    s3_N_values = ro.ravel()[s3_N_indexes] * m2
    s3_N_sum = np.sum(s3_N_values, axis=1)
    s3 = (s3_V_sum * s3_N_sum).reshape(S-4, N-4, N-4)
    
    s4_V_indexes = cell_indeces_flat + np.array([N, -N, 2*N, -2*N])
    s4_V_values = P_p.ravel()[s4_V_indexes] * m1
    s4_V_sum = np.sum(s4_V_values, axis=1)
    s4_N_indexes = cell_indeces_flat + np.array([N, -N])
    s4_N_values = ro.ravel()[s4_N_indexes] * m2
    s4_N_sum = np.sum(s4_N_values, axis=1)
    s4 = (s4_V_sum * s4_N_sum).reshape(S-4, N-4, N-4)

    s5_V_indexes = cell_indeces_flat + np.array([1, -1, 2, -2])
    s5_V_values = P_p.ravel()[s5_V_indexes] * m1
    s5_V_sum = np.sum(s5_V_values, axis=1)
    s5_N_indexes = cell_indeces_flat + np.array([1, -1])
    s5_N_values = ro.ravel()[s5_N_indexes] * m2
    s5_N_sum = np.sum(s5_N_values, axis=1)
    s5 = (s5_V_sum * s5_N_sum).reshape(S-4, N-4, N-4)
    
    Z[2:-2, 2:-2, 2:-2] += (s3 + s4 + s5) * ro[2:-2, 2:-2, 2:-2]
    P[2:-2, 2:-2, 2:-2] -= Z[2:-2, 2:-2, 2:-2] * K_2_by_3[2:-2, 2:-2, 2:-2]
    P[ro < 0.1] = 0
    return P

In [4]:
%lprun -s -f P_step_ravel -T lp_results.txt  P_step_ravel(P_pp, P_p)
%cat lp_results.txt


*** Profile printout saved to text file 'lp_results.txt'. 
Timer unit: 1e-06 s

Total time: 2.79922 s
File: <ipython-input-3-3e336a603436>
Function: P_step_ravel at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def P_step_ravel(P_pp, P_p):
     2                                               '''
     3                                               mb work with flat and then reshape in return
     4                                               norm by now, mb add some more optimisations in future, also cuda
     5                                               '''
     6                                           
     7         1        196.0    196.0      0.0      S = P_p.shape[0]
     8         1          5.0      5.0      0.0      N = P_p.shape[1]
     9                                               
    10         1      15484.0  15484.0      0.6      P = np.zeros_like(P_p)
    11         1      24161.0  2416

In [ ]:


# LOOPS = 1

# t0 = time()
# for i in range(LOOPS):
#     a1 = P_step(P_pp, P_p)
# print(time() - t0)

# t0 = time()
# for i in range(LOOPS):
#     a2 = P_step_ravel(P_pp, P_p)
# print(time() - t0)

# # print(alpha_2)
# print(np.allclose(a1, a2))
# print(a1[30, 30, 30])
# print(a2[30, 30, 30])
# print(np.mean(alpha_1), np.mean(alpha_2), np.max(alpha_2))

